In [1]:
import numpy as np
import pandas as pd
pd.options.display.max_rows = 20
np.random.seed(12345)
import matplotlib.pyplot as plt
plt.rc('figure', figsize=(10, 6))
np.set_printoptions(precision=4, suppress=True)

## 层次化索引
层次化索引（hierarchical indexing）是`pandas`的⼀项重要功能，它使你能在⼀个轴上拥有多个（两个以上）索引级别。抽象点说，它使你能以低维度形式处理⾼维度数据。我们先来看⼀个简单的例⼦：创建⼀个`Series`，并⽤⼀个由列表或数组组成的列表作为索引：

In [2]:
data = pd.Series(np.random.randn(9),
                 index=[['a', 'a', 'a', 'b', 'b', 'c', 'c', 'd', 'd'],
                        [1, 2, 3, 1, 3, 1, 2, 2, 3]])
data

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

看到的结果是经过美化的带有`MultiIndex`索引的`Series`的格式。索引之间的“间隔”表示“直接使⽤上⾯的标签”：

In [3]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

对于⼀个层次化索引的对象，可以使⽤所谓的部分索引，使⽤它选取数据⼦集的操作更简单：

In [4]:
data['b']

1   -0.555730
3    1.965781
dtype: float64

In [5]:
data['b':'c']

b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
dtype: float64

In [6]:
data.loc[['b', 'd']]

b  1   -0.555730
   3    1.965781
d  2    0.281746
   3    0.769023
dtype: float64

有时甚⾄还可以在“内层”中进⾏选取：

In [7]:
data.loc[:, 2]

a    0.478943
c    0.092908
d    0.281746
dtype: float64

层次化索引在数据重塑和基于分组的操作（如透视表⽣成）中扮演着重要的⻆⾊。例如，可以通过`unstack`⽅法将这段数据重新安排到⼀个`DataFrame`中：

In [8]:
data.unstack()

,1,2,3
a,-0.204708,0.478943,-0.519439
b,-0.555730,NaN,1.965781
c,1.393406,0.092908,NaN
d,NaN,0.281746,0.769023


`unstack`的逆运算是`stack`：

In [9]:
data.unstack().stack()

a  1   -0.204708
   2    0.478943
   3   -0.519439
b  1   -0.555730
   3    1.965781
c  1    1.393406
   2    0.092908
d  2    0.281746
   3    0.769023
dtype: float64

对于⼀个`DataFrame`，每条轴都可以有分层索引：

In [10]:
frame = pd.DataFrame(np.arange(12).reshape((4, 3)),
                     index=[['a', 'a', 'b', 'b'], [1, 2, 1, 2]],
                     columns=[['Ohio', 'Ohio', 'Colorado'],
                              ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

各层都可以有名字（可以是字符串，也可以是别的`Python`对象）。如果指定了名称，它们就会显示在控制台输出中：

In [11]:
frame.index.names = ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

有了部分列索引，因此可以轻松选取列分组

In [12]:
frame['Ohio']

color      Green  Red
key1 key2            
a    1         0    1
     2         3    4
b    1         6    7
     2         9   10

可以单独创建`MultiIndex`然后复⽤。上⾯那个`DataFrame`中的（带有分级名称）列可以这样创建：

In [13]:
pd.MultiIndex.from_arrays([['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']],
                       names=['state', 'color'])

MultiIndex([(    'Ohio', 'Green'),
            (    'Ohio',   'Red'),
            ('Colorado', 'Green')],
           names=['state', 'color'])

## 重排与分级排序
有时，你需要重新调整某条轴上各级别的顺序，或根据指定级别上的值对数据进⾏排序。`swaplevel`接受两个级别编号或名称，并返回⼀个互换了级别的新对象（但数据不会发⽣变化）：

In [14]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

⽽`sort_index`则根据单个级别中的值对数据进⾏排序。交换级别时，常常也会⽤到`sort_index`，这样最终结果就是按照指定顺序进⾏字⺟排序了：

In [15]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [16]:
frame.swaplevel(0, 1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

## 根据级别汇总统计
许多对`DataFrame`和`Series`的描述和汇总统计都有⼀个`level`选项，它⽤于指定在某条轴上求和的级别。再以上⾯那个`DataFrame`为例，我们可以根据⾏或列上的级别来进⾏求和：

In [17]:
frame.sum(level='key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [18]:
frame.sum(level='color', axis=1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

## 使用DataFrame的列进行索引
⼈们经常想要将`DataFrame`的⼀个或多个列当做⾏索引来⽤，或者可能希望将⾏索引变成`DataFrame`的列。以下⾯这个`DataFrame`为例：

In [19]:
frame = pd.DataFrame({'a': range(7), 'b': range(7, 0, -1),
                      'c': ['one', 'one', 'one', 'two', 'two',
                            'two', 'two'],
                      'd': [0, 1, 2, 0, 1, 2, 3]})
frame

,a,b,c,d
0,0,7,one,0
1,1,6,one,1
2,2,5,one,2
3,3,4,two,0
4,4,3,two,1
5,5,2,two,2
6,6,1,two,3


`DataFrame`的`set_index`函数会将其⼀个或多个列转换为⾏索引，并创建⼀个新的`DataFrame`：

In [20]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

默认情况下，那些列会从`DataFrame`中移除，但也可以将其保留下来：

In [21]:
frame.set_index(['c', 'd'], drop=False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

`reset_index`的功能跟`set_index`刚好相反，层次化索引的级别会被转移到列⾥⾯：

In [22]:
frame2.reset_index()

,c,d,a,b
0,one,0,0,7
1,one,1,1,6
2,one,2,2,5
3,two,0,3,4
4,two,1,4,3
5,two,2,5,2
6,two,3,6,1


## 合并数据集
`pandas`对象中的数据可以通过⼀些⽅式进⾏合并：
- `pandas.merge`可根据⼀个或多个键将不同`DataFrame`中的⾏连接起来。`SQL`或其他关系型数据库的⽤户对此应该会⽐较熟悉，因为它实现的就是数据库的`join`操作。
- `pandas.concat`可以沿着⼀条轴将多个对象堆叠到⼀起。
- 实例⽅法`combine_first`可以将重复数据编接在⼀起，⽤⼀个对象中的值填充另⼀个对象中的缺失值。

## 数据库风格的DataFrame合并
数据集的合并（merge）或连接（join）运算是通过⼀个或多个键将⾏链接起来的。这些运算是关系型数据库（基于SQL）的核⼼。`pandas`的`merge`函数是对数据应⽤这些算法的主要切⼊点。  
以⼀个简单的例⼦开始：

In [23]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df2 = pd.DataFrame({'key': ['a', 'b', 'd'],
                    'data2': range(3)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,a,5
6,b,6


In [24]:
df2

,key,data2
0,a,0
1,b,1
2,d,2


这是⼀种多对⼀的合并。`df1`中的数据有多个被标记为`a`和`b`的⾏，⽽`df2`中`key`列的每个值则仅对应⼀⾏。对这些对象调⽤`merge`即可得到：

In [25]:
pd.merge(df1, df2)

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


注意，我并没有指明要⽤哪个列进⾏连接。如果没有指定，`merge`就会将重叠列的列名当做键。不过，最好明确指定⼀下：

In [26]:
pd.merge(df1, df2, on='key')

,key,data1,data2
0,b,0,1
1,b,1,1
2,b,6,1
3,a,2,0
4,a,4,0
5,a,5,0


如果两个对象的列名不同，也可以分别进⾏指定：

In [27]:
df3 = pd.DataFrame({'lkey': ['b', 'b', 'a', 'c', 'a', 'a', 'b'],
                    'data1': range(7)})
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'],
                    'data2': range(3)})
pd.merge(df3, df4, left_on='lkey', right_on='rkey')

,lkey,data1,rkey,data2
0,b,0,b,1
1,b,1,b,1
2,b,6,b,1
3,a,2,a,0
4,a,4,a,0
5,a,5,a,0


可能你已经注意到了，结果⾥⾯`c`和`d`以及与之相关的数据消失了。默认情况下，`merge`做的是“内连接”；结果中的键是交集。其他⽅式还有`"left"、"right"`以及`"outer"`。外连接求取的是键的并集，组合了左连接和右连接的效果：

In [28]:
pd.merge(df1, df2, how='outer')

,key,data1,data2
0,b,0.0,1.0
1,b,1.0,1.0
2,b,6.0,1.0
3,a,2.0,0.0
4,a,4.0,0.0
5,a,5.0,0.0
6,c,3.0,NaN
7,d,NaN,2.0


不同的连接类型:
![](./不同的连接类型.jpg)

多对多的合并有些不直观。看下⾯的例⼦：

In [29]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a', 'c', 'a', 'b'],
                    'data1': range(6)})
df2 = pd.DataFrame({'key': ['a', 'b', 'a', 'b', 'd'],
                    'data2': range(5)})
df1

,key,data1
0,b,0
1,b,1
2,a,2
3,c,3
4,a,4
5,b,5


In [30]:
df2

,key,data2
0,a,0
1,b,1
2,a,2
3,b,3
4,d,4


In [31]:
pd.merge(df1, df2, on='key', how='left')

,key,data1,data2
0,b,0,1.0
1,b,0,3.0
2,b,1,1.0
3,b,1,3.0
4,a,2,0.0
5,a,2,2.0
6,c,3,NaN
7,a,4,0.0
8,a,4,2.0
9,b,5,1.0


多对多连接产⽣的是⾏的笛卡尔积。由于左边的`DataFrame`有3个"b"⾏，右边的有2个，所以最终结果中就有6个"b"⾏。连接⽅式只影响出现在结果中的不同的键的值：

In [32]:
pd.merge(df1, df2, how='inner')

,key,data1,data2
0,b,0,1
1,b,0,3
2,b,1,1
3,b,1,3
4,b,5,1
5,b,5,3
6,a,2,0
7,a,2,2
8,a,4,0
9,a,4,2


要根据多个键进⾏合并，传⼊⼀个由列名组成的列表即可：

In [33]:
left = pd.DataFrame({'key1': ['foo', 'foo', 'bar'],
                     'key2': ['one', 'two', 'one'],
                     'lval': [1, 2, 3]})
right = pd.DataFrame({'key1': ['foo', 'foo', 'bar', 'bar'],
                      'key2': ['one', 'one', 'one', 'two'],
                      'rval': [4, 5, 6, 7]})
pd.merge(left, right, on=['key1', 'key2'], how='outer')

,key1,key2,lval,rval
0,foo,one,1.0,4.0
1,foo,one,1.0,5.0
2,foo,two,2.0,NaN
3,bar,one,3.0,6.0
4,bar,two,NaN,7.0


结果中会出现哪些键组合取决于所选的合并⽅式，你可以这样来理解：多个键形成⼀系列元组，并将其当做单个连接键（当然，实际上并不是这么回事）。

注意：在进⾏列－列连接时，`DataFrame`对象中的索引会被丢弃。

对于合并运算需要考虑的最后⼀个问题是对重复列名的处理。虽然你可以⼿⼯处理列名重叠的问题（查看前⾯介绍的重命名轴标签），但`merge`有⼀个更实⽤的`suffixes`选项，⽤于指定附加到左右两个`DataFrame`对象的重叠列名上的字符串：

In [34]:
pd.merge(left, right, on='key1')

,key1,key2_x,lval,key2_y,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


In [35]:
pd.merge(left, right, on='key1', suffixes=('_left', '_right'))

,key1,key2_left,lval,key2_right,rval
0,foo,one,1,one,4
1,foo,one,1,one,5
2,foo,two,2,one,4
3,foo,two,2,one,5
4,bar,one,3,one,6
5,bar,one,3,two,7


`merge`函数的参数:
![](./merge函数的参数.png)  
`indicator`添加特殊的列`_merge`，它可以指明每个⾏的来源，它的值有`left_only`、`right_only`或`both`，根据每⾏的合并数据的来源。

## 索引上的合并
有时候，`DataFrame`中的连接键位于其索引中。在这种情况下，你可以传⼊`left_index=True`或`right_index=True`（或两个都传）以说明索引应该被⽤作连接键：

In [36]:
left1 = pd.DataFrame({'key': ['a', 'b', 'a', 'a', 'b', 'c'],
                      'value': range(6)})
right1 = pd.DataFrame({'group_val': [3.5, 7]}, index=['a', 'b'])
left1

,key,value
0,a,0
1,b,1
2,a,2
3,a,3
4,b,4
5,c,5


In [37]:
right1

,group_val
a,3.5
b,7.0


In [38]:
pd.merge(left1, right1, left_on='key', right_index=True)

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0


由于默认的`merge`⽅法是求取连接键的交集，因此你可以通过外连接的⽅式得到它们的并集：

In [39]:
pd.merge(left1, right1, left_on='key', right_index=True, how='outer')

,key,value,group_val
0,a,0,3.5
2,a,2,3.5
3,a,3,3.5
1,b,1,7.0
4,b,4,7.0
5,c,5,NaN


对于层次化索引的数据，事情就有点复杂了，因为索引的合并默认是多键合并：

In [40]:
lefth = pd.DataFrame({'key1': ['Ohio', 'Ohio', 'Ohio',
                               'Nevada', 'Nevada'],
                      'key2': [2000, 2001, 2002, 2001, 2002],
                      'data': np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6, 2)),
                      index=[['Nevada', 'Nevada', 'Ohio', 'Ohio',
                              'Ohio', 'Ohio'],
                             [2001, 2000, 2000, 2000, 2001, 2002]],
                      columns=['event1', 'event2'])
lefth

,key1,key2,data
0,Ohio,2000,0.0
1,Ohio,2001,1.0
2,Ohio,2002,2.0
3,Nevada,2001,3.0
4,Nevada,2002,4.0


In [41]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

这种情况下，你必须以列表的形式指明⽤作合并键的多个列（注意⽤`how='outer'`对重复索引值的处理）：

In [42]:
pd.merge(lefth, righth, left_on=['key1', 'key2'], right_index=True)

,key1,key2,data,event1,event2
0,Ohio,2000,0.0,4,5
0,Ohio,2000,0.0,6,7
1,Ohio,2001,1.0,8,9
2,Ohio,2002,2.0,10,11
3,Nevada,2001,3.0,0,1


同时使⽤合并双⽅的索引也没问题：

In [43]:
left2 = pd.DataFrame([[1., 2.], [3., 4.], [5., 6.]],
                     index=['a', 'c', 'e'],
                     columns=['Ohio', 'Nevada'])
right2 = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [13, 14]],
                      index=['b', 'c', 'd', 'e'],
                      columns=['Missouri', 'Alabama'])
left2

,Ohio,Nevada
a,1.0,2.0
c,3.0,4.0
e,5.0,6.0


In [44]:
right2

,Missouri,Alabama
b,7.0,8.0
c,9.0,10.0
d,11.0,12.0
e,13.0,14.0


In [45]:
pd.merge(left2, right2, how='outer', left_index=True, right_index=True)

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


`DataFrame`还有⼀个便捷的`join`实例⽅法，它能更为⽅便地实现按索引合并。它还可⽤于合并多个带有相同或相似索引的`DataFrame`对象，但要求没有重叠的列。在上⾯那个例⼦中，我们可以编写：

In [46]:
left2.join(right2, how='outer')

,Ohio,Nevada,Missouri,Alabama
a,1.0,2.0,NaN,NaN
b,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0
d,NaN,NaN,11.0,12.0
e,5.0,6.0,13.0,14.0


因为⼀些历史版本的遗留原因，`DataFrame`的`join`⽅法默认使⽤的是左连接，保留左边表的⾏索引。它还⽀持在调⽤的`DataFrame`的列上，连接传递的`DataFrame`索引：

In [47]:
left1.join(right1, on='key')

,key,value,group_val
0,a,0,3.5
1,b,1,7.0
2,a,2,3.5
3,a,3,3.5
4,b,4,7.0
5,c,5,NaN


最后，对于简单的索引合并，你还可以向`join`传⼊⼀组`DataFrame`，下⼀节会介绍更为通⽤的`concat`函数，也能实现此功能：

In [48]:
another = pd.DataFrame([[7., 8.], [9., 10.], [11., 12.], [16., 17.]],
                       index=['a', 'c', 'e', 'f'],
                       columns=['New York', 'Oregon'])
another

,New York,Oregon
a,7.0,8.0
c,9.0,10.0
e,11.0,12.0
f,16.0,17.0


In [49]:
left2.join([right2, another])

,Ohio,Nevada,Missouri,Alabama,New York,Oregon
a,1.0,2.0,NaN,NaN,7.0,8.0
c,3.0,4.0,9.0,10.0,9.0,10.0
e,5.0,6.0,13.0,14.0,11.0,12.0


## 轴向连接
另⼀种数据合并运算也被称作连接（concatenation）、绑定（binding）或堆叠（stacking）。`NumPy`的`concatenation`函数可以⽤`NumPy`数组来做：

In [50]:
arr = np.arange(12).reshape((3, 4))
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [51]:
np.concatenate([arr, arr], axis=1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

对于`pandas`对象（如`Series`和`DataFrame`），带有标签的轴使你能够进⼀步推⼴数组的连接运算。具体点说，你还需要考虑以下这些东⻄：
- 如果对象在其它轴上的索引不同，我们应该合并这些轴的不同元素还是只使⽤交集？
- 连接的数据集是否需要在结果对象中可识别？
- 连接轴中保存的数据是否需要保留？许多情况下，`DataFrame`默认的整数标签最好在连接时删掉。 

`pandas`的`concat`函数提供了⼀种能够解决这些问题的可靠⽅式。
我将给出⼀些例⼦来讲解其使⽤⽅式。假设有三个没有重叠索引的`Series`：

In [52]:
s1 = pd.Series([0, 1], index=['a', 'b'])
s2 = pd.Series([2, 3, 4], index=['c', 'd', 'e'])
s3 = pd.Series([5, 6], index=['f', 'g'])

对这些对象调⽤`concat`可以将值和索引粘合在⼀起：

In [53]:
pd.concat([s1, s2, s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

默认情况下，`concat`是在`axis=0`上⼯作的，最终产⽣⼀个新的`Series`。如果传⼊`axis=1`，则结果就会变成⼀个`DataFrame`（`axis=1`是列）：

In [54]:
pd.concat([s1, s2, s3], axis=1)

,0,1,2
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


这种情况下，另外的轴上没有重叠，从索引的有序并集（外连接）上就可以看出来。传⼊`join='inner'`即可得到它们的交集：

In [55]:
s4 = pd.concat([s1, s3])
s4

a    0
b    1
f    5
g    6
dtype: int64

In [56]:
pd.concat([s1, s4], axis=1)

,0,1
a,0.0,0
b,1.0,1
f,NaN,5
g,NaN,6


In [57]:
pd.concat([s1, s4], axis=1, join='inner')

,0,1
a,0,0
b,1,1


在这个例⼦中，`f`和`g`标签消失了，是因为使⽤的是`join='inner'`选项。  

如果需要保留来源信息，就可以通过keys参数进行设置。  
假设你想要在连接轴上创建⼀个层次化索引。使⽤`keys`参数即可达到这个⽬的：  

In [58]:
result = pd.concat([s1, s1, s3], keys=['one', 'two', 'three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [59]:
pd.concat([s1, s1, s3])

a    0
b    1
a    0
b    1
f    5
g    6
dtype: int64

In [60]:
s1

a    0
b    1
dtype: int64

In [61]:
s3

f    5
g    6
dtype: int64

In [62]:
result.unstack()

,a,b,f,g
one,0.0,1.0,NaN,NaN
two,0.0,1.0,NaN,NaN
three,NaN,NaN,5.0,6.0


如果沿着`axis=1`对`Series`进⾏合并，则`keys`就会成为`DataFrame`的列头：

In [63]:
pd.concat([s1, s2, s3], axis=1, keys=['one', 'two', 'three'])

,one,two,three
a,0.0,NaN,NaN
b,1.0,NaN,NaN
c,NaN,2.0,NaN
d,NaN,3.0,NaN
e,NaN,4.0,NaN
f,NaN,NaN,5.0
g,NaN,NaN,6.0


同样的逻辑也适⽤于`DataFrame`对象：

In [64]:
df1 = pd.DataFrame(np.arange(6).reshape(3, 2), index=['a', 'b', 'c'],
                   columns=['one', 'two'])
df2 = pd.DataFrame(5 + np.arange(4).reshape(2, 2), index=['a', 'c'],
                   columns=['three', 'four'])
df1

,one,two
a,0,1
b,2,3
c,4,5


In [65]:
df2

,three,four
a,5,6
c,7,8


In [66]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

如果传⼊的不是列表⽽是⼀个字典，则字典的键就会被当做`keys`选项的值：

In [67]:
pd.concat({'level1': df1, 'level2': df2}, axis=1)

level1     level2     
     one two  three four
a      0   1    5.0  6.0
b      2   3    NaN  NaN
c      4   5    7.0  8.0

此外还有两个⽤于管理层次化索引创建⽅式的参数（参⻅表8-3）。  
![](./concat函数的参数.jpg)  
举个例⼦，我们可以⽤`names`参数命名创建的轴级别(`names`参数可以给拼接后形成的数据结构添加名字)：

In [68]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'],
          names=['upper', 'lower'])

upper level1     level2     
lower    one two  three four
a          0   1    5.0  6.0
b          2   3    NaN  NaN
c          4   5    7.0  8.0

最后⼀个关于`DataFrame`的问题是，`DataFrame`的⾏索引不包含任何相关数据：

In [69]:
df1 = pd.DataFrame(np.random.randn(3, 4), columns=['a', 'b', 'c', 'd'])
df2 = pd.DataFrame(np.random.randn(2, 3), columns=['b', 'd', 'a'])
df1

,a,b,c,d
0,1.246435,1.007189,-1.296221,0.274992
1,0.228913,1.352917,0.886429,-2.001637
2,-0.371843,1.669025,-0.438570,-0.539741


In [70]:
df2

,b,d,a
0,0.476985,3.248944,-1.021228
1,-0.577087,0.124121,0.302614


在这种情况下，传⼊`ignore_index=True`即可：

In [71]:
pd.concat([df1, df2], ignore_index=True)

,a,b,c,d
0,1.246435,1.007189,-1.296221,0.274992
1,0.228913,1.352917,0.886429,-2.001637
2,-0.371843,1.669025,-0.438570,-0.539741
3,-1.021228,0.476985,NaN,3.248944
4,0.302614,-0.577087,NaN,0.124121


## 合并重叠数据
还有⼀种数据组合问题不能⽤简单的合并（merge）或连接（concatenation）运算来处理。⽐如说，你可能有索引全部或部分重叠的两个数据集。举个有启发性的例⼦，我们使⽤`NumPy`的`where`函数，它表示⼀种等价于⾯向数组的`if-else`：

In [72]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan],
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(np.arange(len(a), dtype=np.float64),
              index=['f', 'e', 'd', 'c', 'b', 'a'])
b[-1] = np.nan
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [73]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    NaN
dtype: float64

In [74]:
np.where(pd.isnull(a), b, a)

array([0. , 2.5, 2. , 3.5, 4.5, nan])

`Series`有⼀个`combine_first`⽅法，实现的也是⼀样的功能，还带有`pandas`的数据对⻬：

In [75]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

对于`DataFrame`，`combine_first`⾃然也会在列上做同样的事情，因此你可以将其看做：⽤传递对象中的数据为调⽤对象的缺失数据“打补丁”：

In [76]:
df1 = pd.DataFrame({'a': [1., np.nan, 5., np.nan],
                    'b': [np.nan, 2., np.nan, 6.],
                    'c': range(2, 18, 4)})
df2 = pd.DataFrame({'a': [5., 4., np.nan, 3., 7.],
                    'b': [np.nan, 3., 4., 6., 8.]})
df1

,a,b,c
0,1.0,NaN,2
1,NaN,2.0,6
2,5.0,NaN,10
3,NaN,6.0,14


In [77]:
df2

,a,b
0,5.0,NaN
1,4.0,3.0
2,NaN,4.0
3,3.0,6.0
4,7.0,8.0


In [78]:
df1.combine_first(df2)

,a,b,c
0,1.0,NaN,2.0
1,4.0,2.0,6.0
2,5.0,4.0,10.0
3,3.0,6.0,14.0
4,7.0,8.0,NaN


## 重塑和轴向旋转
有许多⽤于重新排列表格型数据的基础运算。这些函数也称作重塑（reshape）或轴向旋转（pivot）运算。

## 重塑层次化索引
层次化索引为`DataFrame`数据的重排任务提供了⼀种具有良好⼀致性的⽅式。主要功能有⼆：
- stack：将数据的列“旋转”为⾏。
- unstack：将数据的⾏“旋转”为列。 

我将通过⼀系列的范例来讲解这些操作。接下来看⼀个简单的`DataFrame`，其中的⾏列索引均为字符串数组：

In [79]:
data = pd.DataFrame(np.arange(6).reshape((2, 3)),
                    index=pd.Index(['Ohio', 'Colorado'], name='state'),
                    columns=pd.Index(['one', 'two', 'three'],
                    name='number'))
data

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


对该数据使⽤`stack`⽅法即可将列转换为⾏，得到⼀个`Series`：

In [80]:
result = data.stack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int32

对于⼀个层次化索引的`Series`，你可以⽤`unstack`将其重排为⼀个`DataFrame`：

In [81]:
result.unstack()

number,one,two,three
state,,,
Ohio,0,1,2
Colorado,3,4,5


默认情况下，`unstack`操作的是最内层（`stack`也是如此）。传⼊分层级别的编号或名称即可对其它级别进⾏`unstack`操作：

In [86]:
result.unstack(0)

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


In [83]:
result.unstack('state')

state,Ohio,Colorado
number,,
one,0,3
two,1,4
three,2,5


如果不是所有的级别值都能在各分组中找到的话，则`unstack`操作可能会引⼊缺失数据：

In [87]:
s1 = pd.Series([0, 1, 2, 3], index=['a', 'b', 'c', 'd'])
s2 = pd.Series([4, 5, 6], index=['c', 'd', 'e'])
data2 = pd.concat([s1, s2], keys=['one', 'two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [88]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


`stack`默认会滤除缺失数据，因此该运算是可逆的：

In [89]:
data2.unstack()

,a,b,c,d,e
one,0.0,1.0,2.0,3.0,NaN
two,NaN,NaN,4.0,5.0,6.0


In [90]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [91]:
data2.unstack().stack(dropna=False)

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
     e    NaN
two  a    NaN
     b    NaN
     c    4.0
     d    5.0
     e    6.0
dtype: float64

在对`DataFrame`进⾏`unstack`操作时，作为旋转轴的级别将会成为结果中的最低级别:

In [92]:
df = pd.DataFrame({'left': result, 'right': result + 5},
                  columns=pd.Index(['left', 'right'], name='side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [93]:
df.unstack('state')

side   left          right         
state  Ohio Colorado  Ohio Colorado
number                             
one       0        3     5        8
two       1        4     6        9
three     2        5     7       10

当调⽤`stack`，我们可以指明轴的名字：

In [94]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

## 将“⻓格式”旋转为“宽格式”
多个时间序列数据通常是以所谓的“⻓格式”（long）或“堆叠格式”（stacked）存储在数据库和CSV中的。我们先加载⼀些示例数据，做⼀些时间序列规整和数据清洗：

In [95]:
data = pd.read_csv('./macrodata.csv')
data.head()

,year,quarter,realgdp,realcons,realinv,realgovt,realdpi,cpi,m1,tbilrate,unemp,pop,infl,realint
0,1959.0,1.0,2710.349,1707.4,286.898,470.045,1886.9,28.98,139.7,2.82,5.8,177.146,0.00,0.00
1,1959.0,2.0,2778.801,1733.7,310.859,481.301,1919.7,29.15,141.7,3.08,5.1,177.830,2.34,0.74
2,1959.0,3.0,2775.488,1751.8,289.226,491.260,1916.4,29.35,140.5,3.82,5.3,178.657,2.74,1.09
3,1959.0,4.0,2785.204,1753.7,299.356,484.052,1931.3,29.37,140.0,4.33,5.6,179.386,0.27,4.06
4,1960.0,1.0,2847.699,1770.5,331.722,462.199,1955.5,29.54,139.6,3.50,5.2,180.007,2.31,1.19


In [97]:
periods = pd.PeriodIndex(year=data.year, quarter=data.quarter,
                         name='date')
columns = pd.Index(['realgdp', 'infl', 'unemp'], name='item')
data = data.reindex(columns=columns)
data.index = periods.to_timestamp('D', 'end')
ldata = data.stack().reset_index().rename(columns={0: 'value'})

这就是多个时间序列（或者其它带有两个或多个键的可观察数据，这⾥，我们的键是date和item）的⻓格式。表中的每⾏代表⼀次观察。  
关系型数据库（如MySQL）中的数据经常都是这样存储的，因为固定架构（即列名和数据类型）有⼀个好处：随着表中数据的添加，`item`列中的值的种类能够增加。在前⾯的例⼦中，`date`和`item`通常就是主键（⽤关系型数据库的说法），不仅提供了关系完整性，⽽且提供了更为简单的查询⽀持。有的情况下，使⽤这样的数据会很麻烦，你可能会更喜欢`DataFrame`，不同的`item`值分别形成⼀列，`date`列中的时间戳则⽤作索引。`DataFrame`的`pivot`⽅法完全可以实现这个转换：

In [98]:
ldata[:10]

,date,item,value
0,1959-03-31 23:59:59.999999999,realgdp,2710.349
1,1959-03-31 23:59:59.999999999,infl,0.000
2,1959-03-31 23:59:59.999999999,unemp,5.800
3,1959-06-30 23:59:59.999999999,realgdp,2778.801
4,1959-06-30 23:59:59.999999999,infl,2.340
5,1959-06-30 23:59:59.999999999,unemp,5.100
6,1959-09-30 23:59:59.999999999,realgdp,2775.488
7,1959-09-30 23:59:59.999999999,infl,2.740
8,1959-09-30 23:59:59.999999999,unemp,5.300
9,1959-12-31 23:59:59.999999999,realgdp,2785.204


In [99]:
pivoted = ldata.pivot('date', 'item', 'value')
pivoted

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2
...,...,...,...
2008-09-30 23:59:59.999999999,-3.16,13324.600,6.0
2008-12-31 23:59:59.999999999,-8.79,13141.920,6.9
2009-03-31 23:59:59.999999999,0.94,12925.410,8.1


前两个传递的值分别⽤作⾏和列索引，最后⼀个可选值则是⽤于填充`DataFrame`的数据列。假设有两个需要同时重塑的数据列：

In [100]:
ldata['value2'] = np.random.randn(len(ldata))
ldata[:10]

,date,item,value,value2
0,1959-03-31 23:59:59.999999999,realgdp,2710.349,0.523772
1,1959-03-31 23:59:59.999999999,infl,0.000,0.000940
2,1959-03-31 23:59:59.999999999,unemp,5.800,1.343810
3,1959-06-30 23:59:59.999999999,realgdp,2778.801,-0.713544
4,1959-06-30 23:59:59.999999999,infl,2.340,-0.831154
5,1959-06-30 23:59:59.999999999,unemp,5.100,-2.370232
6,1959-09-30 23:59:59.999999999,realgdp,2775.488,-1.860761
7,1959-09-30 23:59:59.999999999,infl,2.740,-0.860757
8,1959-09-30 23:59:59.999999999,unemp,5.300,0.560145
9,1959-12-31 23:59:59.999999999,realgdp,2785.204,-1.265934


如果忽略最后⼀个参数，得到的`DataFrame`就会带有层次化的列：

In [102]:
pivoted = ldata.pivot('date', 'item')
pivoted[:5]

value                    value2            \
item                           infl   realgdp unemp      infl   realgdp   
date                                                                      
1959-03-31 23:59:59.999999999  0.00  2710.349   5.8  0.000940  0.523772   
1959-06-30 23:59:59.999999999  2.34  2778.801   5.1 -0.831154 -0.713544   
1959-09-30 23:59:59.999999999  2.74  2775.488   5.3 -0.860757 -1.860761   
1959-12-31 23:59:59.999999999  0.27  2785.204   5.6  0.119827 -1.265934   
1960-03-31 23:59:59.999999999  2.31  2847.699   5.2 -2.359419  0.332883   

                                         
item                              unemp  
date                                     
1959-03-31 23:59:59.999999999  1.343810  
1959-06-30 23:59:59.999999999 -2.370232  
1959-09-30 23:59:59.999999999  0.560145  
1959-12-31 23:59:59.999999999 -1.063512  
1960-03-31 23:59:59.999999999 -0.199543

In [103]:
pivoted['value'][:5]

item,infl,realgdp,unemp
date,,,
1959-03-31 23:59:59.999999999,0.00,2710.349,5.8
1959-06-30 23:59:59.999999999,2.34,2778.801,5.1
1959-09-30 23:59:59.999999999,2.74,2775.488,5.3
1959-12-31 23:59:59.999999999,0.27,2785.204,5.6
1960-03-31 23:59:59.999999999,2.31,2847.699,5.2


注意:`pivot`其实就是⽤`set_index`创建层次化索引，再⽤`unstack`重塑：

## 将“宽格式”旋转为“⻓格式”
旋转`DataFrame`的逆运算是`pandas.melt`。它不是将⼀列转换到多个新的`DataFrame`，⽽是合并多个列成为⼀个，产⽣⼀个⽐输⼊⻓的`DataFrame`。看⼀个例⼦：

In [104]:
df = pd.DataFrame({'key': ['foo', 'bar', 'baz'],
                   'A': [1, 2, 3],
                   'B': [4, 5, 6],
                   'C': [7, 8, 9]})
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9


`key`列可能是分组指标，其它的列是数据值。当使⽤`pandas.melt`，我们必须指明哪些列是分组指标。下⾯使⽤`key`作为唯⼀的分组指标：

In [105]:
melted = pd.melt(df, ['key'])
melted

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6
6,foo,C,7
7,bar,C,8
8,baz,C,9


使⽤`pivot`，可以重塑回原来的样⼦：

In [106]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

variable,A,B,C
key,,,
bar,2,5,8
baz,3,6,9
foo,1,4,7


因为`pivot`的结果从列创建了⼀个索引，⽤作⾏标签，我们可以使⽤`reset_index`将数据移回列：

In [107]:
reshaped.reset_index()

variable,key,A,B,C
0,bar,2,5,8
1,baz,3,6,9
2,foo,1,4,7


你还可以指定列的⼦集，作为值的列：

In [108]:
pd.melt(df, id_vars=['key'], value_vars=['A', 'B'])  # 无C

,key,variable,value
0,foo,A,1
1,bar,A,2
2,baz,A,3
3,foo,B,4
4,bar,B,5
5,baz,B,6


`pandas.melt`也可以不⽤分组指标：

In [110]:
pd.melt(df, value_vars=['A', 'B', 'C'])

,variable,value
0,A,1
1,A,2
2,A,3
3,B,4
4,B,5
5,B,6
6,C,7
7,C,8
8,C,9


In [111]:
pd.melt(df, value_vars=['key', 'A', 'B'])

,variable,value
0,key,foo
1,key,bar
2,key,baz
3,A,1
4,A,2
5,A,3
6,B,4
7,B,5
8,B,6


In [112]:
df

,key,A,B,C
0,foo,1,4,7
1,bar,2,5,8
2,baz,3,6,9
